<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/3_Analisis_y_visualizacion_datos/7_Datos_geograficos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🗺️ **Análisis de datos geográficos**


**Objetivos de Aprendizaje**

Al finalizar este notebook, serás capaz de:

1.  **Comprender la estructura de los datos geográficos vectoriales** (shapefiles) y su relevancia en el análisis económico.
2.  **Navegar el Geoportal del DANE** para descargar el Marco Geoestadístico Nacional, la fuente oficial de los mapas de Colombia.
3.  **Cargar, procesar y visualizar datos geoespaciales** en Python utilizando la librería `geopandas`.
4.  **Enriquecer un mapa** uniendo datos económicos (como el Índice de Pobreza Multidimensional) para crear visualizaciones coropléticas y derivar conclusiones económicas a partir de patrones espaciales.

**Introducción**

Analizar variables económicas sin su contexto espacial es como suponer que todos los agentes económicos operan en un mismo punto, sin vecinos, sin regiones y sin costos de desplazamiento. Hoy vamos a romper ese supuesto. Aprenderemos a incorporar la dimensión espacial para visualizar cómo fenómenos como la pobreza, la riqueza o la actividad económica no se distribuyen de manera uniforme, sino que forman **clústeres**, **corredores** y **fronteras** que revelan historias profundas sobre el desarrollo económico de un país.

## Importar librerías

Lo primero es asegurarnos de tener las herramientas necesarias. La librería principal para este trabajo es `geopandas`, que extiende la funcionalidad de `pandas` para permitir el manejo de datos geográficos.

In [ ]:
# Si no tienes instaladas las librerías, ejecuta esta celda.
# Geopandas puede tener dependencias complejas; la instalación puede tardar unos minutos.
# %pip install geopandas requests
# %pip install requests

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import zipfile
import io
import os

## Mejorar visualización de los dataframes

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

## Cargar el dataset

Para cualquier análisis geoespacial sobre Colombia, la fuente de datos primordial es el **Departamento Administrativo Nacional de Estadística (DANE)**. El DANE mantiene el **Marco Geoestadístico Nacional (MGN)**, que es el conjunto de todas las divisiones geográficas del país (departamentos, municipios, etc.).

**Guía paso a paso para descargar los datos del DANE:**

1.  **Ir al Geoportal del DANE**: Abran un navegador y vayan a [https://geoportal.dane.gov.co/](https://geoportal.dane.gov.co/). Este es el centro de recursos geográficos de la entidad.

2.  **Buscar el Marco Geoestadístico**: En la página principal, en el menú de descargas, busquen la sección del **Marco Geoestadístico Nacional (MGN)**. Suelen actualizarlo cada año.

3.  **Seleccionar la Capa de Interés**: El MGN contiene muchas capas (veredas, sectores, etc.). Para nuestro análisis, nos interesa la capa de **Municipios**. Busquen la opción que diga algo como `Versión MGN2024-Nivel Municipio`.

4.  **Descargar el Shapefile**: Los datos geográficos vectoriales suelen venir en formato **shapefile**. No es un solo archivo, sino un conjunto de archivos (`.shp`, `.shx`, `.dbf`, `.prj`, etc.) que funcionan juntos. Por eso, el DANE siempre los ofrece dentro de un archivo `.zip`.

Para facilitar la clase, el siguiente código descargará y descomprimirá los datos del MGN 2024 directamente por nosotros.

In [ ]:
# URL oficial del DANE para el shapefile de municipios del MGN 2024
url = 'https://geoportal.dane.gov.co/descargas/mgn_2024/MGN2024_MPIO_POLITICO.zip'
#  Definir carpeta local donde se guardará el shapefile después de descargarlo y extraerlo
directorio_extraccion = "./shapefiles_dane"

In [ ]:
print("Descargando y descomprimiendo los datos geográficos del DANE...")
try:
    # Descarga el archivo ZIP desde la URL del DANE
    r = requests.get(url)
    # Convierte el contenido descargado (r.content) en un objeto tipo archivo en memoria con io.BytesIO().
    # Luego abre ese archivo como un ZIP usando zipfile.ZipFile
    z = zipfile.ZipFile(io.BytesIO(r.content))
    # Descomprime todos los archivos dentro del ZIP en la carpeta indicada
    z.extractall(directorio_extraccion)
    print(f"Archivos extraídos en la carpeta: '{directorio_extraccion}'")
except Exception as e:
    print(f"Ocurrió un error en la descarga: {e}")

In [ ]:
# --- Carga robusta del Shapefile ---
# Buscamos el archivo .shp dentro del directorio de extracción para no depender de un nombre fijo
path_to_shp = None
for file in os.listdir(directorio_extraccion):
    if file.endswith(".shp"):
        path_to_shp = os.path.join(directorio_extraccion, file)
        break

if path_to_shp:
    print(f"Shapefile encontrado: {path_to_shp}")
    # Usar GeoPandas (gpd) para leer el shapefile y cargarlo en un GeoDataFrame
    gdf_colombia = gpd.read_file(path_to_shp)
    print("\nPrimeras 5 filas del GeoDataFrame:")
    display(gdf_colombia.head())
else:
    print("No se encontró ningún archivo .shp en el directorio. Verifique la descarga.")

**Nota:** Un GeoDataFrame es como un DataFrame de pandas pero con una columna especial geometry que guarda la geometría de los municipios (polígonos).

## Inspección Inicial del Mapa

Antes de hacer cualquier análisis, es fundamental visualizar los datos geográficos para asegurarse de que se cargaron correctamente.

In [ ]:
# Geopandas tiene un método .plot() integrado, que facilita una visualización rápida.
fig, ax = plt.subplots(figsize=(10, 10))
gdf_colombia.plot(ax=ax, edgecolor='black', linewidth=0.2)

# Ajustes estéticos del gráfico
ax.set_title('Mapa Político de Municipios de Colombia', fontsize=16)
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')
ax.grid(True, linestyle='--', alpha=0.6)
plt.show()

## Enriqueciendo el Mapa con Datos Económicos

Vamos a agregar el **Índice de Pobreza Multidimensional (IPM)** por municipio para 2018 (uno de los últimos datos consolidados a nivel municipal por el DANE). Este índice va más allá del ingreso y considera factores como educación, salud y condiciones de vivienda.

El paso clave aquí es un **`merge`**, similar al que hacemos en `pandas`. La única condición es tener una **columna llave** en común entre el GeoDataFrame (mapa) y el DataFrame (datos económicos). Esta llave será el código único de cada municipio.

In [ ]:
df_ipm = pd.read_csv("Base_municipios.txt", sep="|", decimal=",", converters={'Cod. Municipio': str})
df_ipm.head()

In [ ]:
df_ipm.info()

In [ ]:
gdf_colombia.info()

In [ ]:
df_mapa_ipm = gdf_colombia.merge(df_ipm[['Cod. Municipio', '% pobreza municipio']], left_on='mpio_cdpmp', right_on='Cod. Municipio', how='left')
df_mapa_ipm

## Creación de un Mapa Coroplético

Ahora que tenemos los datos y las geometrías en un solo lugar, podemos crear un **mapa coroplético**. Este tipo de mapa utiliza una escala de colores para representar la intensidad de una variable en diferentes áreas, permitiéndonos identificar patrones espaciales de un vistazo.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 15))

df_mapa_ipm.plot(column='% pobreza municipio',           # La columna cuyos valores determinarán el color
                    ax=ax,                 # El eje sobre el cual graficar
                    legend=True,           # Mostrar la barra de colores (leyenda)
                    cmap='viridis_r',      # Paleta de colores. '_r' la invierte (amarillo=bajo, morado=alto)
                    missing_kwds={         # Cómo mostrar municipios sin datos de IPM
                        'color': 'lightgrey',
                        'label': 'Datos no disponibles',
                    },
                    legend_kwds={          # Opciones para la leyenda
                        'label': "Índice de Pobreza Multidimensional (IPM) 2018",
                        'orientation': "horizontal"
                    })

ax.set_title('Distribución de la Pobreza Multidimensional en Colombia (2018)', fontdict={'fontsize': 20, 'fontweight': 'bold'})
ax.set_axis_off() # Quitamos los ejes de latitud y longitud que no aportan información aquí
plt.show()

**Análisis Económico-Espacial:** El mapa revela una fuerte 'autocorrelación espacial positiva' de la pobreza. Los municipios con alto IPM tienden a estar rodeados de otros municipios con alto IPM. Vemos una clara concentración de pobreza en la periferia del país (costa Pacífica, Caribe y la Orinoquía), en contraste con el centro andino. Esta visualización es una poderosa herramienta para el diseño de políticas públicas focalizadas.

## Ejercicio

El departamento de Antioquia es conocido por su fuerte economía pero también por sus desigualdades internas. Filtra el GeoDataFrame `df_mapa_ipm` para quedarte únicamente con los municipios de **Antioquia** (código de departamento `dpto_ccdgo` es `'05'`) y crea un mapa coroplético del IPM solo para esa región. ¿Qué patrones observas? ¿Qué municipio es el más y menos pobre?